In [1]:
import pandas as pd
import re
import random
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./data/train.csv')
train.tail()

,idx,class,conversation
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...
3949,3949,직장 내 괴롭힘 대화,희정씨\n네?\n주말에 시간이 넘쳐나나봐\n갑자기 왜그러세요?\n손이 빤짝빤짝 네일...


### class별 문장 확인

In [3]:
print(train['conversation'][0]) # 협박

지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한번만 도와주세요.
 그냥 다 죽여버려야겠군. 이의 없지?
 제발 도와주세요.


In [4]:
print(train['conversation'][2]) # 기타 괴롭힘

너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.
그만해. 니들 놀리는거 재미없어.
지영아 너가 키 160이지? 그럼 재는 160도 안돼는거네?
너 군대도 안가고 좋겠다.
니들이 나 작은데 보태준거 있냐?
난쟁이들도 장가가고하던데. 너도 희망을 가져봐 
더이상 하지마라. 
그 키크는 수술도 있대잖아? 니네 엄마는 그거 안해주디?
나람 해줬어. 저 키로 어찌살아.
제발 그만 괴롭히라고!


In [5]:
print(train['conversation'][3]) # 갈취

어이 거기
예??
너 말이야 너. 이리 오라고
무슨 일.
너 옷 좋아보인다?
얘 돈 좀 있나봐
아니에요.돈 없어요
뒤져서 나오면 넌 죽는다
오늘 피시방 콜?
콜. 마지막 기회다. 있는거 다 내놔
정말 없어요


In [6]:
print(train['conversation'][3947]) # 직장 내 괴롭힘

나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.
왜 그래. 들리겠어.
들리라고 해. 아니 일 못해 눈치 없어. 본인도 슬슬 깨달아야 되지 않나?
하긴 그래서 저번에 부장님도.
그렇다니깐! 부장님이 진짜 착하신 분인데 오죽 답답했으면 말을 했겠어?
그러게.
회사생활 같이 한게 몇년인데 대답 하나 제대로 네 하고 싹싹하게 하는걸 못봤어.
나도 그건 못본거 같다.
지금도 그래. 야 야! 너 바로 앞에서 얘기하는데 귀가 먹어서 안들리는거니 안들리는척 하는거니? 뭐 말 좀 해봐?
야 너 왜 그래.
말씀이 너무 심하신거 아니에요?


In [7]:
train.isna().sum()

idx             0
class           0
conversation    0
dtype: int64

### 정규화

In [8]:
from soynlp.normalizer import *

In [9]:
def preprocess_sentence(sentence):
    emoticon_normalize(sentence) # soynlp의 ㅋㅋㅋ, ㅠㅠㅠ 등 처리 메소드
    repeat_normalize(sentence) # soynlp 한글 반복 처리 메소드
    sentence = re.sub(r'([^ㄱ-ㅎ가-힣])', " ", sentence) # 한글 제외 모두 공백 처리
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

In [10]:
# 문법기호도 모두 제거
train_list = []
for sentence in train['conversation']:
    train_list.append(preprocess_sentence(sentence))

In [11]:
train_list[0]

'지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해 진짜 죽여버리고 싶게 정말 잘못했습니다 너가 선택해 너가 죽을래 네 가족을 죽여줄까 죄송합니다 정말 잘못했습니다 너에게는 선택권이 없어 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야 선택 못하겠습니다 한번만 도와주세요 그냥 다 죽여버려야겠군 이의 없지 제발 도와주세요'

### augmentation 전에 train, validation split

In [12]:
x_train, x_valid, y_train, y_valid = train_test_split(train_list, 
                                                      train['class'], 
                                                      test_size=0.2, 
                                                      shuffle=True, 
                                                      stratify=train['class'], 
                                                      random_state=34)

### augmentation

In [13]:
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words



def EDA(sentence, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	words = sentence.split(' ')
	words = [word for word in words if word != ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/2) + 1

	n_rs = max(1, int(alpha_rs*num_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [sentence for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [14]:
train_a = []
for sen in x_train:
    aug = EDA(sen, num_aug=9)
    train_a.append(aug)

In [15]:
train_aug = []
for aug in train_a:
    for a in aug:
        train_aug.append(a)

In [16]:
train_aug

['보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 업무 아닌가요 제가 좀 바빠서요 이것 해줄수 있죠 저도 중요한 미팅이 있어서 바쁜데 하아 보미씨 사람 다 바빠요 바쁜거 아니잖아요 신입이라 잘 모르나보네 대리님 그럼 내일까지 완성해서 포워딩할께요 내일까지 지금 당장 지금이요 아까 말씀드렸지만 중요한 미팅이 있어요 미팅은 오늘 야근해서라도 이것부터 끝내주세요 이거 얼마나 걸린다고 제가 입사한지 안되서 업무속도가 느려요 일단 알겠습니다',
 '보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 대리님 업무 아닌가요 제가 좀 바빠서요 이것 좀 해줄수 있죠 저도 중요한 미팅이 있어서 많이 바쁜데 하아 보미씨 여기있는 사람 다 바빠요 보미씨만 바쁜거 아니잖아요 신입이라 잘 모르나보네 대리님 그럼 내일까지 완성해서 포워딩할께요 내일까지 지금 당장 해주세요 지금이요 아까 말씀드렸지만 중요한 미팅이 있어요 미팅은 내일이잖아요 오늘 야근해서라도 이것부터 끝내주세요 이거 얼마나 걸린다고 제가 입사한지 얼마 안되서 업무속도가 느려요 일단 알겠습니다',
 '보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 대리님 아닌가요 좀 바빠서요 이것 좀 해줄수 있죠 중요한 미팅이 있어서 많이 하아 보미씨 여기있는 사람 모두 다 보미씨만 아니잖아요 신입이라 잘 모르나보네 대리님 그럼 내일까지 완성해서 포워딩할께요 내일까지 아니요 지금 당장 해주세요 지금이요 아까 말씀드렸지만 미팅이 있어요 미팅은 내일이잖아요 야근해서라도 끝내주세요 이거 얼마나 걸린다고 제가 입사한지 얼마 안되서 업무속도가 느려요 일단',
 '보미씨 제가 대리님 보냈는데 확인 좀 해주세요 대리님 이건 메일 업무 아닌가요 제가 좀 보미씨 이것 좀 해줄수 있죠 저도 중요한 미팅이 있어서 많이 바쁜데 하아 바빠서요 오늘 신입이라 모두 다 바빠요 보미씨만 바쁜거 아니잖아요 사람 잘 모르나보네 대리님 그럼 얼마나 완성해서 포워딩할께요 내일까지 아니요 지금 당장 해주세요 지금이요 아까 말씀드렸지만 중요한 미팅이 있어

### 증식된만큼 label 맞춰주기

In [17]:
label = list(y_train)

In [18]:
label

['직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '갈취 대화',
 '갈취 대화',
 '직장 내 괴롭힘 대화',
 '갈취 대화',
 '협박 대화',
 '협박 대화',
 '갈취 대화',
 '협박 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '협박 대화',
 '갈취 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '갈취 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '갈취 대화',
 '협박 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '갈취 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '갈취 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '갈취 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '갈취 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '기타 괴롭힘 대화',
 '협박 대화',
 '직장 내 괴롭힘 대화',
 '협박 대화',
 '갈취 대화',
 '협박 대화',
 '갈취 대화',
 '협박 대

In [19]:
aug_label = []

for l in label:
    for i in range(10):
        aug_label.append(l)

In [20]:
aug_label

['직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '기타 괴롭힘 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '직장 내 괴롭힘 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '협박 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',
 '갈취 대화',


In [21]:
train_df = pd.DataFrame({'conversation':train_aug})

In [22]:
train_df['class'] = aug_label

In [23]:
train_df

,conversation,class
0,보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 업무 아닌가요 제가 좀 바...,직장 내 괴롭힘 대화
1,보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 대리님 업무 아닌가요 제가...,직장 내 괴롭힘 대화
2,보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 대리님 아닌가요 좀 바빠서...,직장 내 괴롭힘 대화
3,보미씨 제가 대리님 보냈는데 확인 좀 해주세요 대리님 이건 메일 업무 아닌가요 제가...,직장 내 괴롭힘 대화
4,보미씨 방금 메일 보냈는데 확인 좀 해주세요 대리님 이건 대리님 업무 아닌가요 제가...,직장 내 괴롭힘 대화
...,...,...
31595,야 살고있어 엄마가 너 그렇게 낳아서 안 슬퍼 내가 뭐 어때서 난 괜찮아 제대로 걷...,기타 괴롭힘 대화
31596,야 너네 엄마가 너 그렇게 낳아서 안 슬퍼 어때서 난 제대로 걷지도 병신이 뭐가 괜...,기타 괴롭힘 대화
31597,야 너네 너 그렇게 낳아서 안 슬퍼 내가 뭐 어때서 난 괜찮아 제대로 걷지도 못하는...,기타 괴롭힘 대화
31598,야 너네 엄마가 너 애들을 낳아서 자살했어 슬퍼 내가 뭐 나름 난 괜찮아 제대로 걷...,기타 괴롭힘 대화


In [24]:
df_val = pd.DataFrame({'class': y_valid, 'conversation': x_valid})

### 전처리된 파일 내보내기

In [25]:
train_df.to_csv('./data/train_aug10.csv', index=False)
df_val.to_csv('./data/val.csv', index=False)